In [1]:
import os
import glob
from keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow import keras
from pprint import pprint
# from tensorflow.keras.utils import print_summary
from keras_video import VideoFrameGenerator
from resnet_convlstm import ResNet
import tensorflow as tf
from datetime import datetime as dt
from tensorflow.keras import backend as K
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [2]:
# use sub directories names as classes
from tensorflow_core.python.keras.optimizer_v2.adam import Adam
classes = [i.split(os.path.sep)[1] for i in glob.glob('D:/Datasets/CelebFOM_dataset/*')]
classes.sort()

In [3]:
# some global params
SIZE = (128, 128)
CHANNELS = 3
NBFRAME = 2
BS = 1

In [4]:
# pattern to get videos and classes
glob_pattern='D:/Datasets/CelebFOM_dataset/{classname}/*/*.mp4'
# glob_pattern='D:/Datasets/CelebFOM_dataset/real/*/*.mp4'
# pprint(glob.glob(glob_pattern))

In [5]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=5,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

In [6]:
class_weights={0:3.9945,1:1}
print(train.classes)

In [7]:
# model=keras.applications.Xception(input_shape=(128,128,3),classes=2,include_top=True,weights=None)
# model.summary()
# 
# model.compile(optimizer='nadam',
#                   loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
#                   metrics=['accuracy'])


model=ResNet(input_shape=(5,128, 128, 3), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
model.summary()

In [8]:
# print_summary(model, line_length=150, positions=None, print_fn=None)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])

In [9]:
# model.summary()
# print_summary(model, line_length=150, positions=None, print_fn=None)

In [10]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="XCE_celebFOM_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("MI4_DF+FS_train.csv", append=True, separator=',')

In [ ]:
# model.fit_generator(train)
class_weights={0:3.9945,1:1}
# model.fit_generator(generator=train,epochs=150,
#                     callbacks=callbacks_list,shuffle=False,class_weight=class_weights,verbose=1)

# total_train=240000/8 #30,000
# total_val=40000/8   #5000
model.fit(train,
          verbose=1,
          # steps_per_epoch=int(total_train/batch_size),
          epochs=50,
          # validation_data=validation_generator,
          # validation_steps = int(total_val/batch_size),
          callbacks=[model_checkpoint_callback,csv_logger],
          class_weight=class_weights)